In [1]:
import os, fnmatch, sys

In [5]:
sys.path.insert(1, '..')
import src.authorwordbag

In [10]:
K = src.authorwordbag.WordBagOfAuthorClassifier('..\\books', 50)
print(K.getAuthors())
print(K.classify('rzekł'))

..\books
anything
['adam-mickiewicz', 'boleslaw-prus', 'eliza-orzeszkowa', 'henryk-sienkiewicz', 'stanislaw-wyspianski', 'stefan-zeromski', 'tadeusz-boy-zelenski', 'zgromadzenie-ogolne-onz']
henryk-sienkiewicz


In [9]:
import pathlib
pathlib.Path().resolve()

WindowsPath('C:/Users/mihawb/OneDrive/studia/sem III 2021Z/PRAD/priad-project/misc')